[Group By: split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
planets = sns.load_dataset('planets')

# example 1

In [19]:
# 众数
df = pd.DataFrame({'a':['A','A','A','A','B','B','B','B','B','B'],'b':[1,1,2,3,1,6,2,2,3,3]})
df['c'] = np.random.randint(1,10,10)
df['d'] = [1,1,2,2,3,4,5,6,7,8]
df['e'] = [111]*5 + [222]*5
# df.groupby('a').agg(lambda x: x.value_counts().index[0]).reset_index()

In [28]:
df.groupby('a')[['b','c']].agg(lambda x: tuple(pd.Series.mode(x)))

,b,c
a,,
A,"(1,)","(2, 3, 4, 9)"
B,"(2, 3)","(4,)"


# example 2

In [36]:
def MakeList_1(x):
    """ This function is used to aggregate data that needs to be kept distinc within multi day 
        observations for later use and transformation. It makes a list of the data and if the list is of length 1
        then there is only one line/day observation in that group so the single element of the list is returned. 
        If the list is longer than one then there are multiple line/day observations and the list itself is 
        returned."""
    L = x.tolist()
    if len(L) > 1:
        return L
    else:
        return L[0]


def MakeList_2(x):
    """
    Sometimes, it will raise an exception for the function MakeList above, 
    use tuple type instead of list type to save the result will fix the bug.
    """
    T = tuple(x)
    if len(T) > 1:
        return T
    else:
        return T[0]


DF = pd.DataFrame({
    'date': [
        '2013-04-02', '2013-04-02', '2013-04-02', '2013-04-02', '2013-04-02',
        '2013-04-02', '2013-04-02', '2013-04-02', '2013-04-02', '2013-04-02'
    ],
    'line_code': [
        '401101', '401101', '401102', '401103', '401104', '401105', '401105',
        '401106', '401106', '401107'
    ],
    's.m.v.':
    [7.760, 25.564, 25.564, 9.550, 4.870, 7.760, 25.564, 5.282, 25.564, 5.282]
})
DFGrouped = DF.groupby(['date', 'line_code'], as_index=False)
DF_Agg = DFGrouped.agg({'s.m.v.': MakeList_2})
DF_Agg

,date,line_code,s.m.v.
0,2013-04-02,401101,"(7.76, 25.564)"
1,2013-04-02,401102,25.564
2,2013-04-02,401103,9.55
3,2013-04-02,401104,4.87
4,2013-04-02,401105,"(7.76, 25.564)"
5,2013-04-02,401106,"(5.282, 25.564)"
6,2013-04-02,401107,5.282


# example 3

In [88]:
dic_ = {'A':[1,1,1,2,2,2],'B':np.random.rand(6),'C':['what','what','fuck','what','is','?']}
df= pd.DataFrame(dic_)
df

,A,B,C
0,1,0.615899,what
1,1,0.415562,what
2,1,0.482182,fuck
3,2,0.485147,what
4,2,0.055657,is
5,2,0.794917,?


In [89]:
df.groupby('A')['C'].unique()

A
1     [what, fuck]
2    [what, is, ?]
Name: C, dtype: object

In [90]:
df.groupby('A').sum() # 对group直接调用 sum()会自动过滤掉不是数值型的列

,B
A,
1,1.513643
2,1.335720


In [91]:
df.groupby('A').apply(lambda x: x.sum()) # apply 之后再调用sum则不会

,A,B,C
A,,,
1,3,1.513643,whatwhatfuck
2,6,1.335720,whatis?


In [92]:
df.groupby('A')['C'].apply(lambda x: "{%s}" % ', '.join(x))

A
1    {what, what, fuck}
2         {what, is, ?}
Name: C, dtype: object

In [93]:
def f(x):
    return pd.Series(dict(A = x['A'].sum(), 
                          B = x['B'].sum(), 
                          C = "{%s}" % ', '.join(x['C'])))

df.groupby('A').apply(f)

,A,B,C
A,,,
1,3,1.513643,"{what, what, fuck}"
2,6,1.335720,"{what, is, ?}"


In [94]:
df.groupby('A')['B'].apply(list)

A
1    [0.6158989161161725, 0.4155620260413999, 0.482...
2    [0.4851466990747386, 0.05565665982997248, 0.79...
Name: B, dtype: object

In [95]:
df.groupby('A')['B'].apply(set)

A
1    {0.6158989161161725, 0.4155620260413999, 0.482...
2    {0.4851466990747386, 0.05565665982997248, 0.79...
Name: B, dtype: object

In [96]:
df.groupby('A')['B'].apply(np.sum)

A
1    1.513643
2    1.335720
Name: B, dtype: float64

In [97]:
df.groupby('A')['C'].agg(lambda col: '-'.join(col))

A
1    what-what-fuck
2         what-is-?
Name: C, dtype: object

In [98]:
df.groupby('A').agg({'B':'sum','C':'-'.join})

,B,C
A,,
1,1.513643,what-what-fuck
2,1.335720,what-is-?


In [99]:
df.groupby('A').agg(B_sum=('B','sum'), C=('C', list)).reset_index()

,A,B_sum,C
0,1,1.513643,"[what, what, fuck]"
1,2,1.335720,"[what, is, ?]"


In [102]:
unique_chars = lambda x: '-'.join(x.unique())
df.groupby('A').agg({'C': unique_chars})

,C
A,
1,what-fuck
2,what-is-?


# example 4

In [37]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'C', 'A', 'B', 'C','C','C', 'A', 'B', 'C'],
                   'data1': range(10),
                   'data2': rng.randint(0, 10, 10)},
                   columns = ['key', 'data1', 'data2'])
df = df.sort_values(by='key').reset_index(drop=True)
df

,key,data1,data2
0,A,0,5
1,A,2,3
2,A,7,5
3,B,3,3
4,B,8,2
5,C,1,0
6,C,4,7
7,C,5,9
8,C,6,3
9,C,9,4


In [3]:
df.groupby('key',as_index=False)['key'].agg({'key_count':'count'})

,key,key_count
0,A,3
1,B,2
2,C,5


In [4]:
df.groupby('key',as_index=False)['key'].agg({'key_count':'count','key_sum':'sum'})

,key,key_count,key_sum
0,A,3,AAA
1,B,2,BB
2,C,5,CCCCC


In [8]:
df.groupby('key')['data1'].transform('sum')

0     9
1     9
2     9
3    11
4    11
5    25
6    25
7    25
8    25
9    25
Name: data1, dtype: int64

In [15]:
df['key1'] = df['key'].apply(lambda x: x + '666')
df['key2']  = np.random.rand(10)
df['key2'] = df['key2'].apply(lambda x: str(x))
df

,key,data1,data2,key1,key2
0,A,0,5,A666,0.9794903071662613
1,A,2,3,A666,0.45841106673932785
2,A,7,5,A666,0.7042927697544479
3,B,3,3,B666,0.1293367890574305
4,B,8,2,B666,0.15211636188596134
5,C,1,0,C666,0.8680359841793047
6,C,4,7,C666,0.735274578831239
7,C,5,9,C666,0.3648644316931744
8,C,6,3,C666,0.6173878116906303
9,C,9,4,C666,0.12947451849074054


In [31]:
def myagg(x):
    return x.sum()
df.groupby('key').apply(lambda x: myagg(x))

,key,data1,data2,key1,key2
key,,,,,
A,AAA,9,13,A666A666A666,0.97949030716626130.458411066739327850.7042927...
B,BB,11,5,B666B666,0.12933678905743050.15211636188596134
C,CCCCC,25,23,C666C666C666C666C666,0.86803598417930470.7352745788312390.364864431...


In [17]:
df.groupby('key').apply(lambda x: x.sum())

,key,data1,data2,key1,key2
key,,,,,
A,AAA,9,13,A666A666A666,0.97949030716626130.458411066739327850.7042927...
B,BB,11,5,B666B666,0.12933678905743050.15211636188596134
C,CCCCC,25,23,C666C666C666C666C666,0.86803598417930470.7352745788312390.364864431...


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
key      10 non-null object
data1    10 non-null int64
data2    10 non-null int64
key1     10 non-null object
key2     10 non-null object
dtypes: int64(2), object(3)
memory usage: 528.0+ bytes


In [10]:
df.groupby('key').describe()

data1                                           data2                      \
    count mean       std  min   25%  50%   75%  max count      mean       std   
key                                                                             
A     3.0  3.0  3.605551  0.0  1.00  2.0  4.50  7.0   3.0  4.333333  1.154701   
B     2.0  5.5  3.535534  3.0  4.25  5.5  6.75  8.0   2.0  2.500000  0.707107   
C     5.0  5.0  2.915476  1.0  4.00  5.0  6.00  9.0   5.0  4.600000  3.507136   

                                
     min   25%  50%   75%  max  
key                             
A    3.0  4.00  5.0  5.00  5.0  
B    2.0  2.25  2.5  2.75  3.0  
C    0.0  3.00  4.0  7.00  9.0

In [11]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,3,3
C,1,9


In [54]:
# 聚合并rename
df.groupby('key').agg([np.sum, np.mean, np.std]).rename(columns={'sum': 'my_sum',
                                                                 'mean': 'my_mean',
                                                                 'std': 'my_std'})


data1                    data2                    
    my_sum my_mean    my_std my_sum   my_mean    my_std
key                                                    
A        9     3.0  3.605551     13  4.333333  1.154701
B       11     5.5  3.535534      5  2.500000  0.707107
C       25     5.0  2.915476     23  4.600000  3.507136

In [56]:
# 添加匿名函数
df.groupby('key')['data1'].agg([lambda x: x.max() - x.min(),
                  lambda x: x.median() - x.mean()])


,<lambda_0>,<lambda_1>
key,,
A,7,-1
B,5,0
C,8,0


## NamedAgg

In [57]:
animals = pd.DataFrame({'kind': ['cat', 'dog', 'cat', 'dog'],
                        'height': [9.1, 6.0, 9.5, 34.0],
                        'weight': [7.9, 7.5, 9.9, 198.0]})

animals

,kind,height,weight
0,cat,9.1,7.9
1,dog,6.0,7.5
2,cat,9.5,9.9
3,dog,34.0,198.0


In [65]:
animals.groupby("kind").agg(
    YourNamedCol=pd.NamedAgg(column='height', aggfunc=np.min),
    min_height=pd.NamedAgg(column='height', aggfunc='min'),
    max_height=pd.NamedAgg(column='height', aggfunc='max'),
    std_height=pd.NamedAgg(column='weight', aggfunc='std'),
    average_weight=pd.NamedAgg(column='weight', aggfunc=np.mean))

,YourNamedCol,min_height,max_height,std_height,average_weight
kind,,,,,
cat,9.1,9.1,9.5,1.414214,8.90
dog,6.0,6.0,34.0,134.703842,102.75


In [68]:
# pandas.NamedAgg is just a namedtuple. Plain tuples are allowed as well.
animals.groupby("kind").agg(
    YourNamedCol=('height', np.min),
    min_height=('height', 'min'),
    max_height=('height', 'max'),
    std_height=('weight', 'std'),
    average_weight=('weight', np.mean))

,YourNamedCol,min_height,max_height,std_height,average_weight
kind,,,,,
cat,9.1,9.1,9.5,1.414214,8.90
dog,6.0,6.0,34.0,134.703842,102.75


In [73]:
# 如果想命令的列名非Python支持的变量名：
animals.groupby("kind").agg(**{'我是列名': pd.NamedAgg(column='height', aggfunc=max)})

,我是列名
kind,
cat,9.5
dog,34.0


In [74]:
# 如果是Series groupby aggregations，则只需要写一个执行函数
animals.groupby("kind").height.agg(
    min_height='min',
    max_height='max')

,min_height,max_height
kind,,
cat,9.1,9.5
dog,6.0,34.0


In [76]:
# 这种方式将过时：
animals.groupby("kind").height.agg({'min_height':'min','max_height':'max'})

/Users/zhangqi977/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


,min_height,max_height
kind,,
cat,9.1,9.5
dog,6.0,34.0


In [83]:
# 这么写又可以了
animals.groupby("kind",as_index=False).height.agg({'min_height':'min','max_height':'max'})

,kind,min_height,max_height
0,cat,9.1,9.5
1,dog,6.0,34.0


## filter

In [16]:
# 根据条件 filter，满足条件的组会被保留。key=B的组不满足条件，会被过滤掉
def filter_func(x):
    return x['data2'].max() >= 5

df.groupby('key').filter(filter_func)

,key,data1,data2
0,A,0,5
1,A,2,3
2,A,7,5
5,C,1,0
6,C,4,7
7,C,5,9
8,C,6,3
9,C,9,4


## transform

In [3]:
df.head()

,key,data1,data2
0,A,0,5
1,C,1,0
2,A,2,3
3,B,3,3
4,C,4,7


In [7]:
df.groupby('key')[['data1']].transform(lambda x: x - x.mean())

,data1
0,-3.0
1,-4.0
2,-1.0
3,-2.5
4,-1.0
5,0.0
6,1.0
7,4.0
8,2.5
9,4.0


In [2]:
# transform 方法，对group做一些操作然后再合并，返回的df长度与原始的一样
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-3.0,0.666667
1,-4.0,-4.600000
2,-1.0,-1.333333
3,-2.5,0.500000
4,-1.0,2.400000
5,0.0,4.400000
6,1.0,-1.600000
7,4.0,0.666667
8,2.5,-0.500000
9,4.0,-0.600000


In [4]:
df.groupby('key').mean()

,data1,data2
key,,
A,3.0,4.333333
B,5.5,2.500000
C,5.0,4.600000


In [5]:
df.groupby('key').transform(lambda x: x.mean()) # .transform('mean')

,data1,data2
0,3.0,4.333333
1,5.0,4.600000
2,3.0,4.333333
3,5.5,2.500000
4,5.0,4.600000
5,5.0,4.600000
6,5.0,4.600000
7,3.0,4.333333
8,5.5,2.500000
9,5.0,4.600000


In [40]:
df

,key,data1,data2
0,A,0,5
1,C,1,0
2,A,2,3
3,B,3,3
4,C,4,7
5,C,5,9
6,C,6,3
7,A,7,5
8,B,8,2
9,C,9,4


In [51]:
df.groupby('key').transform(lambda x: x.rank(ascending=True,method='first')) # 排序

,data1,data2
0,1.0,2.0
1,1.0,1.0
2,2.0,1.0
3,1.0,2.0
4,2.0,4.0
5,3.0,5.0
6,4.0,2.0
7,3.0,3.0
8,2.0,1.0
9,5.0,3.0


In [43]:
normalized = (df['data1'] - df.groupby('key')['data1'].transform('mean')) / df.groupby('key')['data1'].transform('std')
normalized

0   -0.832050
1   -1.371989
2   -0.277350
3   -0.707107
4   -0.342997
5    0.000000
6    0.342997
7    1.109400
8    0.707107
9    1.371989
Name: data1, dtype: float64

In [44]:
def normalize(x):
    return (x - x.mean()) / x.std()

df.groupby('key')['data1'].transform(normalize)

0   -0.832050
1   -1.371989
2   -0.277350
3   -0.707107
4   -0.342997
5    0.000000
6    0.342997
7    1.109400
8    0.707107
9    1.371989
Name: data1, dtype: float64

In [45]:
df.groupby('key')['data1'].apply(normalize)

0   -0.832050
1   -1.371989
2   -0.277350
3   -0.707107
4   -0.342997
5    0.000000
6    0.342997
7    1.109400
8    0.707107
9    1.371989
Name: data1, dtype: float64

## apply

In [26]:
# apply，对每个组进行自定义的函数操作
def norm_data1_by_data2_max(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].max()
    # x['data2'] /= x['data1'].max() # 如果执行这个操作，会在前一步操作的结果上进行，而非原始的data1取值
    return x

df.groupby('key').apply(norm_data1_by_data2_max)

,key,data1,data2
0,A,0.000000,5
1,A,0.400000,3
2,A,1.400000,5
3,B,1.000000,3
4,B,2.666667,2
5,C,0.111111,0
6,C,0.444444,7
7,C,0.555556,9
8,C,0.666667,3
9,C,1.000000,4


In [28]:
# 对key分组然后再聚合
df2 = df.set_index('key')
mapping = {'A': 'A', 'B': 'B+C', 'C': 'B+C'}
df2.groupby(mapping).max()

,data1,data2
A,7,5
B+C,9,9


# 例子


In [30]:
planets

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


In [34]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
